# txt 파일 생성

In [ ]:
import json
import os
from glob import glob
from PIL import Image
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torchsummary

%matplotlib inline

In [ ]:
# json 파일경로 한 리스트에 정리

json_path = '../data/temp_datasets/fa/'
json_list = glob(json_path +  '*.json')
json_list

### 1) 전처리 함수

In [ ]:
# 렉트좌표 있는 것만 리스트로 뽑기
from tqdm.notebook import tqdm


check_json_list = []

def rect_check(json_list):
    for each_json in tqdm(json_list):
        with open(each_json, 'r') as f:
            json_info = json.load(f)
    
        rect_values = json_info['데이터셋 정보']['데이터셋 상세설명']['렉트좌표'].values()
        #print(rect_values)
        for each in rect_values:
            if len(each[0]) >= 4 :
                check_json_list.append(each_json)
                break

                
rect_check(json_list)
check_json_list

In [ ]:
len(check_json_list), len(json_list)

In [ ]:
# 상의, 원피스, 아우터 렉트 좌표 있는 것만 리스트로 뽑기
check_json_list2 = []

def bottom_check(json_list):
    for each_json in tqdm(json_list):
        with open(each_json, 'r') as f:
            json_info = json.load(f)
            cate = ['상의', '원피스', '아우터']
            for each_cate in cate:
                if len(json_info['데이터셋 정보']['데이터셋 상세설명']['렉트좌표'][each_cate][0]) >= 4:
                    #print(json_info['데이터셋 정보']['데이터셋 상세설명']['렉트좌표'])
                    check_json_list2.append(each_json)


bottom_check(check_json_list)
check_json_list2

In [ ]:
len(check_json_list2), len(check_json_list)

In [ ]:
# 소매기장 데이터 있는 것만 리스트로 뽑기
check_json_list3 = []

def length_check(json_list):
    for each_json in tqdm(json_list):
        with open(each_json, 'r') as f:
            json_info = json.load(f)
            #print(each_json)

            #label_key_list = list(json_info['데이터셋 정보']['데이터셋 상세설명']['라벨링'].keys())
            cate = ['상의', '원피스', '아우터']
            for each_cate in cate:
                label_value_list = list(json_info['데이터셋 정보']['데이터셋 상세설명']['라벨링'][each_cate][0])
                if '소매기장' in label_value_list :
                        check_json_list3.append(each_json)

length_check(check_json_list2)                

# 중복값 제거
check_json_list3 =  list(set(check_json_list3)) 


In [ ]:
len(check_json_list3), len(check_json_list2)

---

- 데이터 리스트 txt로 임시 저장

In [ ]:
# 라벨링 데이터 리스트 txt로 임시 저장

print(len(check_json_list4))

filePath = '../data/temp_datasets/label.txt'

f = open(filePath, 'w')
for each in check_json_list3:
    print(each)
    f.write(each + '\n')
f.close()


In [ ]:
# 저장한 json 경로 txt 파일 이용

check_json_list3 = []
f = open('../data/temp_datasets/length.txt', 'r')
lines = f.readlines()
for line in lines:
    check_json_list3.append(line.strip())
f.close()



---

### 2) 소매기장 모델링을 위한 추가 전처리 함수

In [ ]:
# 라벨링에 필요한 데이터 구하기

# 소매기장 key가 존재하는 경우, 렉트 좌표가 존재하는 경우, 소매기장 value가 '없음'이 아닌 경우,  라벨링 진행 ★★★

def get_data(json_info):
      
    image_name = json_info['데이터셋 정보']['파일 번호']
    image_width = json_info['이미지 정보']['이미지 너비']
    image_height = json_info['이미지 정보']['이미지 높이']
            
    cate = ['상의', '원피스', '아우터']
    image_label = ''
    box = []
    for each_cate in cate:
        
        # 소매기장 key가 존재하는 경우
        if '소매기장' in json_info['데이터셋 정보']['데이터셋 상세설명']['라벨링'][each_cate][0]:
            
            image_label_txt = json_info['데이터셋 정보']['데이터셋 상세설명']['라벨링'][each_cate][0]['소매기장']
            rects = json_info['데이터셋 정보']['데이터셋 상세설명']['렉트좌표'][each_cate][0]

            # 렉트 좌표가 존재하는 경우
            if len(rects) >= 4:            
                
                # 소매기장 value가 '없음'이 아닌 경우
                
                #if image_label_txt == '없음':
                #    print(image_name, image_label_txt)
                if image_label_txt != '없음':
                    
                    
                    # 라벨링 진행
                    label_list = {0:'민소매', 1: '반팔', 2: '캡', 3:'7부소매', 4: '긴팔'}
                    for idx, (key, value) in enumerate(label_list.items()):
                        if value == image_label_txt:
                            image_label = key
                    
                    box = list(rects.values())   
                    return image_name, image_width, image_height, image_label, box
        

### 3) yolo 형식에 맞도록 좌표 설정 및 txt 파일 생성, 저장 함수

In [ ]:
def data_save(image_name, image_width, image_height, image_label, x1, y1, x2, y2):
    
    # yolo bbox 변환
    # xmin, ymin, xmax, ymax
    #x1, y1, x2, y2 = box
    
    x_center = (x1 + x2/2) / image_width  # 중앙값을 구해서 x1을 더해야 한다 ★★★
    y_center = (y1 + y2/2) / image_height
    width = x2 / image_width
    height = y2 / image_height

    #print(image_name, x_center, y_center, width,height)
    file = open('../data/temp_datasets/label/' + str(image_name) + '.txt', 'w')
    file.write(f'{image_label} {x_center:f} {y_center:f} {width:f} {height:f}\n')
    file.close()            


### 4) json 파일별 txt 생성 함수

In [ ]:
check_json_list4 = []

def change_data(json_list):
    for each_json in tqdm(json_list):
        with open(each_json, 'r') as f:
            json_info = json.load(f)
            #print(json_info['데이터셋 정보']['파일 번호'])
            #print(len(json_info['데이터셋 정보']['데이터셋 상세설명']['렉트좌표']['하의'][0]))
            print(get_data(json_info))
            try:
                image_name, image_width, image_height, image_label, box = get_data(json_info)
                #print(image_name, image_width, image_height, image_label, box)

                x1, y1, x2, y2 = box     
                
                data_save(image_name, image_width, image_height, image_label, x1, y1, x2, y2)
                check_json_list4.append(each_json)
            except:
                pass



### 5) txt 생성 

In [ ]:
# 소매기장 key가 존재하는 경우, 렉트 좌표가 존재하는 경우, 소매기장 value가 '없음'이 아닌 경우,  
# 라벨링 진행해서 txt 저장
# 라벨링 파일 리스트도 저장 - label


change_data(check_json_list3)

In [ ]:
# 라벨링 데이터 리스트 txt로 저장

print(len(check_json_list4))

filePath = '../data/temp_datasets/label.txt'

f = open(filePath, 'w')
for each in check_json_list4:
    print(each)
    f.write(each + '\n')
f.close()
